In [16]:
from random import random
from math import exp
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs)]} for i in range(n_hidden)]
    output_layer = [{'weights':[random() for i in range(n_hidden)]} for i in range(n_outputs)]
    network.append(hidden_layer)
    network.append(output_layer)
    return network
network = initialize_network(2,1,2)

In [14]:
def out_calculate(weights,inputs):
    net = 0
    for i in range(len(weights)):
        net+=weights[i]*inputs[i]
    return net

def activation(net):
    return 1.0/(1.0+exp(-net))

def forward_propagation(network,row):
    inputs = row
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activate = out_calculate(neuron['weights'],inputs)
            neuron['output']=activation(activate)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs
forward_propagation(network,[1,0])

In [18]:
network

[[{'weights': [0.9455097802353822, 0.044681922267236995],
   'output': 0.7202112626171201}],
 [{'weights': [0.39967036411143275], 'output': 0.5714689852661156},
  {'weights': [0.08500348828734006], 'output': 0.5153003389681584}]]

In [41]:
expected = [0,1]
def back_propagate(network, expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i!=len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i+1]:
                    error+=(neuron['weights'][j]*neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output']-expected[j])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta']=errors[j]*neuron['output']*(1-neuron['output'])
back_propagate(network,expected)

In [42]:
network

[[{'weights': [0.9455097802353822, 0.044681922267236995],
   'output': 0.7202112626171201,
   'delta': 0.008428527735978651}],
 [{'weights': [0.29887803092932],
   'output': 0.5714689852661156,
   'delta': 0.139948287972964},
  {'weights': [0.17219330580389636],
   'output': 0.5153003389681584,
   'delta': -0.12106144688674257}]]

In [43]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs= row
        if i!=0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]-=l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']

                
update_weights(network,[1,0],1)                

In [44]:
network

[[{'weights': [0.9370812524994035, 0.036253394531258344],
   'output': 0.7202112626171201,
   'delta': 0.008428527735978651}],
 [{'weights': [0.05813740977424331],
   'output': 0.5714689852661156,
   'delta': 0.139948287972964},
  {'weights': [0.38044457020719524],
   'output': 0.5153003389681584,
   'delta': -0.12106144688674257}]]

In [ ]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))